In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

### Q1. What are the top 5 brands by receipts scanned for most recent month?

In [2]:
# Q1
database = "fetchrewards.db"
conn = create_connection(database)
cur = conn.cursor()
cur.execute("SELECT i.brandCode , strftime('%m',datetime(r.dateScanned/1000,'unixepoch')), COUNT(*) \
             FROM scanned_item i \
             LEFT JOIN receipts r   \
             ON i.receipt_id=r._id \
             WHERE i.brandCode IS NOT NULL  AND \
             strftime('%m',datetime(r.dateScanned/1000,'unixepoch'))=( \
             SELECT  strftime('%m',(max(datetime(r.dateScanned/1000,'unixepoch')))) \
             from scanned_item i \
             LEFT JOIN receipts r \
             ON i.receipt_id=r._id \
             WHERE i.brandCode IS NOT NULL)\
             GROUP BY i.brandCode \
             ORDER BY COUNT(*) DESC")
rows = cur.fetchall()
conn.close()

In [3]:
#result: 
#There are only top 3 brands for most recent month. There are 'BRAND','MISSION' and 'VIVA'
rows

[('BRAND', '02', 3), ('MISSION', '02', 2), ('VIVA', '02', 1)]

### Q2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [4]:
# Q2: previous month is 01
database = "fetchrewards.db"
conn = create_connection(database)
cur = conn.cursor()
cur.execute("with temp(code, count, rank) as \
            (SELECT i.brandCode , COUNT(*) , RANK() OVER (ORDER BY COUNT(*) DESC) \
             FROM scanned_item i \
             LEFT JOIN receipts r   \
             ON i.receipt_id=r._id \
             WHERE i.brandCode IS NOT NULL   AND \
             strftime('%m',datetime(r.dateScanned/1000,'unixepoch'))=( \
             SELECT  strftime('%m', date(max(datetime(r.dateScanned/1000,'unixepoch')), '-1 months')) \
             from scanned_item i \
             LEFT JOIN receipts r \
             ON i.receipt_id=r._id \
             WHERE i.brandCode IS NOT NULL) \
             GROUP BY i.brandCode \
             ORDER BY COUNT(*) DESC) \
             select * from temp \
             where code in ('BRAND','MISSION','VIVA')")
rows = cur.fetchall()
conn.close()

In [5]:
#result
#The rank of the brands('BRAND','MISSION') are 26 and 35 respectively in the previous month. 
#There is no receipt that contain any item produced by'VIVA' in the previous month. 
rows

[('BRAND', 19, 26), ('MISSION', 16, 35)]

### Q3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [6]:
# Q3: Assume status, "Finished" is equal to status, "Accepted".
database = "fetchrewards.db"
conn = create_connection(database)
cur = conn.cursor()
cur.execute("SELECT avg(totalSpent) \
             from receipts \
             WHERE rewardsReceiptStatus='FINISHED'")
rows1 = cur.fetchall()

cur.execute("SELECT avg(totalSpent) \
             from receipts \
             WHERE rewardsReceiptStatus='REJECTED'")
rows2 = cur.fetchall()

conn.close()

In [7]:
#result
#The average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ is 80.85
rows1

[(80.85430501930502,)]

In [8]:
#result
#The average spend from receipts with 'rewardsReceiptStatus’ of ‘Rejected’ is 23.33
rows2

[(23.326056338028184,)]

### Q4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [9]:
# Q4 Assume status, "Finished" is equal to status, "Accepted".
database = "fetchrewards.db"
conn = create_connection(database)
cur = conn.cursor()
cur.execute("SELECT SUM(purchasedItemCount) \
             from receipts \
             WHERE rewardsReceiptStatus='FINISHED'")
rows1 = cur.fetchall()

cur.execute("SELECT SUM(purchasedItemCount) \
             from receipts \
             WHERE rewardsReceiptStatus='REJECTED'")
rows2 = cur.fetchall()
conn.close()

In [10]:
#result
#The total number of item purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ is 8184
rows1

[(8184.0,)]

In [11]:
#The total number of item purchased from receipts with 'rewardsReceiptStatus’ of ‘Rejected’ is 173
rows2

[(173.0,)]

### Q5. Which brand has the most spend among users who were created within the past 6 months?

In [12]:
#Q5

database = "fetchrewards.db"
conn = create_connection(database)
cur = conn.cursor()
cur.execute("SELECT i.brandCode,SUM(r.totalSpent), u._id \
             FROM scanned_item i \
             LEFT JOIN receipts r   \
             ON i.receipt_id=r._id \
             LEFT JOIN users u\
             on r.userId=u._id \
             where u._id in \
            (SELECT _id \
             FROM USERS \
             WHERE date(datetime(createdDate/1000,'unixepoch')) > \
            (select date(max(datetime(createdDate/1000,'unixepoch')), '-6 months') \
             FROM USERS )) AND\
             i.brandCode IS NOT NULL \
             GROUP BY i.brandCode \
             ORDER BY SUM(r.totalSpent) desc limit 1")
rows = cur.fetchall()
conn.close()

In [13]:
#result
#'BEN AND JERRYS' is the brand that has  the most spend among users who were created within the past 6 months
rows

[('BEN AND JERRYS', 197337.68000000023, '60023de5fb296c121a81b955')]

### Q6. Which brand has the most transactions among users who were created within the past 6 months?

In [14]:
#Q6 

database = "fetchrewards.db"
conn = create_connection(database)
cur = conn.cursor()
cur.execute("SELECT i.brandCode, u._id, COUNT(*) \
            FROM scanned_item i \
            LEFT JOIN receipts r   \
            ON i.receipt_id=r._id \
            LEFT JOIN users u\
            on r.userId=u._id \
            where u._id in \
            (SELECT _id \
             FROM USERS \
             WHERE date(datetime(createdDate/1000,'unixepoch')) > \
            (select date(max(datetime(createdDate/1000,'unixepoch')), '-6 months') \
             FROM USERS )) AND\
             i.brandCode IS NOT NULL \
            GROUP BY i.brandCode \
            ORDER BY COUNT(*) DESC limit 1")
rows = cur.fetchall()
conn.close()

In [15]:
#result
#'HY-VEE' is the brand that has  the most spend among users who were created within the past 6 months
rows

[('HY-VEE', '6000b75bbe5fc96dfee1d4d3', 291)]